<a href="https://colab.research.google.com/github/hfarley/bigdata-finalproject/blob/main/BigDataFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [19]:
#install spark. we are using the one that uses hadoop as the underlying scheduler.
# !rm spark-3.2.4-bin-hadoop3.2.tgz 
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf  spark-3.2.4-bin-hadoop3.2.tgz
!ls -l

#Provides findspark.init() to make pyspark importable as a regular library.
os.environ["SPARK_HOME"] = "spark-3.2.4-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

total 590204
drwxr-xr-x  2 root root      4096 Apr 27 00:11 incidents_10000.parquet
-rw-r--r--  1 root root      1584 Apr 27 00:19 month_incidents.py
-rw-r--r--  1 root root   1975841 Apr 27 00:09 nfd_incidents_xd_seg.parquet
drwxr-xr-x  1 root root      4096 Apr 25 13:34 sample_data
drwxr-xr-x 13 1000 1000      4096 Apr  9 21:17 spark-3.2.4-bin-hadoop3.2
-rw-r--r--  1 root root 301183180 Apr  9 21:35 spark-3.2.4-bin-hadoop3.2.tgz
-rw-r--r--  1 root root 301183180 Apr  9 21:35 spark-3.2.4-bin-hadoop3.2.tgz.1


In [20]:
!pip install -q findspark pyspark
import findspark
findspark.init()

In [ ]:
# 1. query any month of any year within the database and it will show a map of the incidents during that month. 
# 2. input a road segment, and it will show a graph of the density of the times 
# that incidents occur in that road segment. 
# 3. query the average response time for each road segment, and sort it to find
#  which road segments have the highest and lowest response times. 



In [ ]:
#this creates a smaller sample dataset to work with locally

from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("read_parquet").getOrCreate()

df = spark.read.parquet("nfd_incidents_xd_seg.parquet")

# create a smaller DataFrame with the first 1000 entries
df_small = df.limit(10000)

# write the output 
df_small.write.parquet("incidents_10000.parquet")

Notes: 


*   .show() will give the first 20 entries
*   in the spark assignment we only used SparkContext, but here we also need Spark Session

For the first task we need ID, latitude & longitude (or, alternatively geometry), and one of the time variables - I decided to use the local time variable since that would be more meaningful for someone trying to visualize the data

We also add columns for month and year (based on time_local) for querying purposes

* Need to decide how we want to visualize this and get the coordinates to draw a map of Davidson county which then shows the incidents
* Month & year are specified as command line arguments



In [15]:
%%file month_incidents.py


import argparse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month

if __name__ == '__main__':

  parser = argparse.ArgumentParser(description='Process some strings.')
  parser.add_argument('--monthArg', type=int, choices=range(1,13), help='month (1-12)', required=True)
  parser.add_argument('--yearArg', type=int, choices=range(2017,2023), help='year (2017-2022)', required=True)
  args = parser.parse_args()

  # replace this line with the s3 pass when testing over EMR
  conf = SparkConf().setAppName('month_incidents').set('spark.hadoop.validateOutputSpecs', False)
  sc = SparkContext(conf=conf).getOrCreate()
  spark = SparkSession(sc)

  
  try:

    incidents = spark.read.parquet("incidents_10000.parquet")

    incidents = incidents.drop("emdCardNumber", "time_utc", "response_time_sec", "day_of_week", "weekend_or_not", "Incident_ID", "Dist_to_Seg", "XDSegID")

    incidents = incidents.withColumn("year", year(incidents["time_local"]))
    incidents = incidents.withColumn("month", month(incidents["time_local"]))
    
    filtered_incidents = incidents.filter((incidents.year == args.yearArg) & (incidents.month == args.monthArg))
    
    filtered_incidents.show()
    
    # counts.repartition(1).saveAsTextFile("")

  finally:
    # very important: stop the context. Otherwise you may get an error that context is still alive. if you are on colab just restart the runtime if you face problem
    #finally is used to make sure the context is stopped even with errors
    sc.stop()

  pass

Overwriting month_incidents.py


In [ ]:
#to test locally
!spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 month_incidents.py --monthArg 2 --yearArg 2017

In profs sample code they use plotly express, but: 


*   plotly express can only be used with pandas, which means we have to convert our spark df to a pandas df to use plotly express but obviously this is an issue with the size of our dataset
*   to use regular plotly, we have to sign up/create an account to get an access key for their API calls to create the maps we need



In [ ]:
%%file road_density_incidents.py

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

if __name__ == '__main__':

  # replace this line with the s3 pass when testing over EMR
  conf = SparkConf().setAppName('month_incidents').set('spark.hadoop.validateOutputSpecs', False)
  sc = SparkContext(conf=conf).getOrCreate()
  spark = SparkSession(sc)

  
  try:
  

  finally:
    sc.stop()

  pass

In [ ]:
!spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 road_density_incidents.py

In [24]:
%%file road_avg_response.py

# 3. query the average response time for each road segment, and sort it to find
#  which road segments have the highest and lowest response times. 

from pyspark import SparkContext, SparkConf
from operator import add
from pyspark.sql import SparkSession


if __name__ == '__main__':

  # replace this line with the s3 pass when testing over EMR
  conf = SparkConf().setAppName('month_incidents').set('spark.hadoop.validateOutputSpecs', False)
  sc = SparkContext(conf=conf).getOrCreate()
  spark = SparkSession(sc)

  
  try:

    incidents = spark.read.parquet("incidents_10000.parquet")

    incidents = incidents.drop("latitude", "longitude", "emdCardNumber", "time_utc", "day_of_week", "weekend_or_not", "Incident_ID", "geometry")

    
    incidents.show()
  

  finally:
    sc.stop()

  pass

Overwriting road_avg_response.py


In [25]:
!spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 road_avg_response.py

:: loading settings :: url = jar:file:/content/spark-3.2.4-bin-hadoop3.2/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-8_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-05d8d868-e698-4ff0-b04b-1a86e0ffd750;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-8_2.11;2.4.7 in central
	found org.apache.kafka#kafka_2.11;0.8.2.1 in central
	found org.scala-lang.modules#scala-xml_2.11;1.0.2 in central
	found com.yammer.metrics#metrics-core;2.2.0 in central
	found org.slf4j#slf4j-api;1.7.16 in central
	found org.scala-lang.modules#scala-parser-combinators_2.11;1.1.0 in central
	found com.101tec#zkclient;0.3 in central
	found log4j#log4j;1.2.17 in central
	found org.apache.kafka#kafka-clients;0.8.2.1 in central
	found net.jpountz.lz4#lz4;1.2.0 in central
	found org.xerial.snapp